### IMPORTACIONES

In [2]:
import pandas as pd
import json
import ast
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
warnings.filterwarnings('ignore')

#### ETL SOBRE USERS_ITEMS

In [8]:


users_items = 'datos_json/australian_users_items.json'

listado = []

with open(users_items, 'r', encoding= 'utf-8') as archivo:
    for linea in archivo.readlines():
        listado.append(ast.literal_eval(linea))

dfUserItems = pd.DataFrame(listado)



In [9]:
#Guardo en formato csv para mejorar tiempos de carga y flujo de trabajo.
dfUserItems.to_csv('json_a_csv/australian_users_items.csv', index=False)

In [10]:
#Cargo el csv previamente creado.
userItems = pd.read_csv("json_a_csv/australian_users_items.csv")

userItems

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [11]:
userItems.isnull().any()

user_id        False
items_count    False
steam_id       False
user_url       False
items          False
dtype: bool

In [12]:
from ast import literal_eval

userItems['items'] = userItems['items'].apply(literal_eval)

In [13]:
userItems[['items'][0]]

0        [{'item_id': '10', 'item_name': 'Counter-Strik...
1        [{'item_id': '10', 'item_name': 'Counter-Strik...
2        [{'item_id': '1200', 'item_name': 'Red Orchest...
3        [{'item_id': '10', 'item_name': 'Counter-Strik...
4        [{'item_id': '300', 'item_name': 'Day of Defea...
                               ...                        
88305    [{'item_id': '413850', 'item_name': 'CS:GO Pla...
88306    [{'item_id': '11020', 'item_name': 'TrackMania...
88307                                                   []
88308    [{'item_id': '304930', 'item_name': 'Unturned'...
88309                                                   []
Name: items, Length: 88310, dtype: object

In [14]:
filasItems = userItems.explode('items')

columnasItems = pd.json_normalize(filasItems['items'])

filasItems = filasItems.reset_index(drop=True)

userItems2 = filasItems.join(columnasItems)

In [15]:
#Dropeo 
userItemsFinal = userItems2.drop(['items', 'user_url'], axis=1)

In [16]:
userItemsFinal

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,519140,Minds Eyes,3.0,3.0


In [17]:
userItemsFinal[['item_id']]

,item_id
0,10
1,20
2,30
3,40
4,50
...,...
5170010,373330
5170011,388490
5170012,521570
5170013,519140


In [18]:
userItemsFinal.dtypes

user_id              object
items_count           int64
steam_id              int64
item_id              object
item_name            object
playtime_forever    float64
playtime_2weeks     float64
dtype: object

In [19]:
userItemsFinal.isnull().any()

user_id             False
items_count         False
steam_id            False
item_id              True
item_name            True
playtime_forever     True
playtime_2weeks      True
dtype: bool

In [20]:
itemsNulos = userItemsFinal[userItemsFinal['item_id'].isna()].index

userItemsFinal.iloc[itemsNulos]

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
3733,Wackky,0,76561198039117046,NaN,NaN,NaN,NaN
3849,76561198079601835,0,76561198079601835,NaN,NaN,NaN,NaN
6019,hellom8o,0,76561198117222320,NaN,NaN,NaN,NaN
6523,starkillershadow553,0,76561198059648579,NaN,NaN,NaN,NaN
7237,darkenkane,0,76561198058876001,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5169470,76561198316380182,0,76561198316380182,NaN,NaN,NaN,NaN
5169471,76561198316970597,0,76561198316970597,NaN,NaN,NaN,NaN
5169472,76561198318100691,0,76561198318100691,NaN,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,0,76561198328759259,NaN,NaN,NaN,NaN


In [21]:
useritemsdropeados = userItemsFinal.dropna()

In [22]:
#tomo un usuario y verifico que se hayan eliminado los registros nulos
userItemsFinal[userItemsFinal['user_id']=='bokkkbokkk']

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
75761,bokkkbokkk,0,76561198006988360,NaN,NaN,NaN,NaN
75762,bokkkbokkk,0,76561198006988360,NaN,NaN,NaN,NaN


In [23]:
#tomo un usuario y verifico que se hayan eliminado los registros nulos
useritemsdropeados[useritemsdropeados['user_id']=='bokkkbokkk']

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks


In [24]:
useritemsdropeados.isna().mean()

user_id             0.0
items_count         0.0
steam_id            0.0
item_id             0.0
item_name           0.0
playtime_forever    0.0
playtime_2weeks     0.0
dtype: float64

In [25]:
useritemsdropeados

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5170009,76561198329548331,7,76561198329548331,346330,BrainBread 2,0.0,0.0
5170010,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0


In [26]:
useritemsdropeados.rename(columns={'item_id': 'id'}, inplace=True)

In [27]:
useritemsdropeados


,user_id,items_count,steam_id,id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5170009,76561198329548331,7,76561198329548331,346330,BrainBread 2,0.0,0.0
5170010,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0


In [28]:
useritemsdropeados.to_csv('csv_limpios/user_items.csv', index= False)

In [8]:
# revision del archivo parquet, problemas con el tipo de la columna id, lo casteo a int porque lo tomaba como object
ui = pd.read_parquet('datos_parquet/user_items.parquet')



In [9]:
ui['id'] = ui['id'].astype(int)

In [7]:
ui.dtypes

user_id              object
items_count           int64
steam_id              int64
id                    int32
item_name            object
playtime_forever    float64
playtime_2weeks     float64
dtype: object

In [ ]:
# Se convierte el DataFrame a una tabla de Arrow para posteriormente comprimirlo en parquet
table = pa.Table.from_pandas(ui)

# Especifica el nombre del archivo Parquet y comprime con snappy (puedes elegir otros algoritmos)
parquet_file = 'datos_parquet/user_items.parquet'
pq.write_table(table, parquet_file, compression='snappy')